In [ ]:
# Importamos JSON
import json

# Guardamos las credenciales para consumir api de twitter (sacadas de apps.twiiter.com)
credentials = {
    "KEY": "c0Jlf7B9Ez56i6rsnBSeiBCii",
    "KEY_SECRET": "yuCAV2TdGrvitWBGQondppKaxNlQfnrc5krRCYvNpu3miY2gQQ",
    "TOKEN": "1279067386520440833-YLZvyZzPDZlQ3bGa5SH2fhugVPl4Cs",
    "ACCESS_SECRET": "DZuy6rzCJFZ4723jVl2gITgf2jCXr1cqlFRtQXFytrIKD"
}

credentials_PRO = {
    "KEY": "XqNe5tbaycDxrrIOVqHR1NoPk",
    "KEY_SECRET": "wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q",
    "TOKEN": "1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ",
    "ACCESS_SECRET": "rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM"
}

credentials
credentials_PRO

In [ ]:
# Intslamos Twythom
!pip install twython

In [ ]:
# Instalamos geopy
!pip install geopy

In [ ]:
#importamos las librerias necesarias
import pandas as pd

### consultar api tiempo real
from twython import TwythonStreamer

### Se guardara en un archivo .csv
import csv

## imortar libreria para buscar coordenadas
from geopy.geocoders import Nominatim



In [ ]:
## EJEMPLO de como encontrra las coordenadas sabiendo el pais y/o ciudad
geolocator = Nominatim(user_agent="BD-Project-JJ")

coordinates =  []

location = geolocator.geocode("Port Moody, British Columbia")

if location:
    coordinates.append(location.latitude)
    coordinates.append(location.longitude)
else:
    no_location += 1
    coordinates.append(0)
    coordinates.append(0)
coordinates
        

In [ ]:
geolocator = Nominatim(user_agent="BD-Project-JJ")


#procesar los datos con filtro
def process_tweet(tweet):
    # titulos
    #date,hashtags,text,user,verifieduser,registeredon,followersuser,followinguser,tweetsuser,favoritestweetsuser,retweet,quote,reply,likes,isquote,originaldate,originaltext,originaluser,originalverified,originallocation,originalquotecount,originalreplycount,originalretweetcount,originalfavoritecount,language,device,location,latitude,longitude

        d = {}
        d['date'] = tweet['created_at']
        d["hashtags"] = [hashtag["text"] for hashtag in tweet['entities']['hashtags']]
        d['text'] = tweet['text']
        #datos user
        d['user'] = tweet['user']['screen_name']
        d['verified'] = tweet['user']['verified']
        d['created'] = tweet['user']['created_at']
        d['followers'] = tweet['user']['followers_count']
        d['following'] = tweet['user']['friends_count']
        d['tweets'] = tweet['user']['statuses_count']
        d['Favorites'] = tweet['user']['favourites_count']
        
        # datos tweet
        d['retweet'] = tweet['retweet_count']
        d['quote'] = tweet['quote_count']
        d['reply'] = tweet['reply_count'] 
        d['like'] = tweet['favorite_count']

        # Si tiene un tweet original
        d['isQuote'] = tweet['is_quote_status']
        if tweet['is_quote_status']:
            d['originalTweetDate'] = tweet['quoted_status']['created_at']
            d['originalTweetText'] = tweet['quoted_status']['text']
            d['originalTweetUser'] = tweet['quoted_status']['user']['screen_name']
            d['originalTweetVerified'] = tweet['quoted_status']['user']['verified']
            d['originalTweetLocation'] = tweet['quoted_status']['user']['location']
            d['originalTweetQuote'] = tweet['quoted_status']['quote_count']
            d['originalTweetReply'] = tweet['quoted_status']['reply_count']
            d['originalTweetRetweet'] = tweet['quoted_status']['retweet_count']
            d['originalTweetFavorite'] = tweet['quoted_status']['favorite_count']
        else:
            d['originalTweetDate'] = "None"
            d['originalTweetText'] = "None"
            d['originalTweetUser'] = "None"
            d['originalTweetVerified'] = "None"
            d['originalTweetLocation'] = "None"
            d['originalTweetQuote'] = "None"
            d['originalTweetReply'] = "None"
            d['originalTweetRetweet'] = "None"
            d['originalTweetFavorite'] = "None"
            
                
        d['language'] = tweet['lang']

        ## guardar dispositivo
        if "android" in tweet['source']:
            d['device'] = "ANDROID"
        elif "iphone" in tweet['source']:
            d['device'] = "IPHONE"
        elif "ipad" in tweet['source']:
            d['device'] = "IPAD"
        elif "Web App" in tweet['source']:
            d['device'] = "WEB APP"
        else:
            d['device'] = "OTHER"


        d['user_location'] = tweet['user']['location']
        ## guardar coordenadas ubicacion
        if tweet['user']['location']:
            coordinates = search_coordinates(tweet['user']['location'])
            d['latitude'] = coordinates[0]
            d['longitude'] = coordinates[1]
        else:
            d['latitude'] = "None"
            d['longitude'] = "None"


        return d

def search_coordinates(location):
    #Buscar y guardar coordenadas
    coordinates = []

    location = geolocator.geocode(location)

    if location:
        coordinates.append(location.latitude)
        coordinates.append(location.longitude)
    else:
        coordinates.append("None")
        coordinates.append("None")
    return coordinates

# Heredando clase
#se le pasa las credenciales (probar siempre con credentials normales.)
class Streamer(TwythonStreamer):

    #Datos recibidos
    def on_success(self, data):
        #si se necesita poner un condicional
        #if data['atributo'] == 'condicion':
        #print(data)
        tweet_data = process_tweet(data)
        self.save_tweet(tweet_data)

    #Error
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    #Guardar en un archivo el resultados
    def save_tweet(self, tweet):
        #reempalzar si no esta trabajando en visualstudio o no esta en la carpeta origen del proyecto
        #with open(r'tweets.csv', 'a', encoding="utf-8") as file: 
        with open(r'/home/jovyan/work/slides/data/tweets.csv', 'a', encoding="utf-8") as file:
            write = csv.writer(file)
            write.writerow(list(tweet.values()))

In [ ]:
### Recoleccion de datos

In [ ]:

# Intanciar
#stream =  Streamer(credentials['KEY'], credentials['KEY_SECRET'], credentials['TOKEN'], credentials['ACCESS_SECRET'])
stream =  Streamer(credentials_PRO['KEY'], credentials_PRO['KEY_SECRET'], credentials_PRO['TOKEN'], credentials_PRO['ACCESS_SECRET'])

#Iniciar consulta en tiempo real
stream.statuses.filter(track="covid")
# Para poder para la consulta y luego ver los resultados debe parar el proceso manualmente, es decir interrumpirlo.
# dando click en el boton de interrupt IPython kernel

In [ ]:
tweets = pd.read_csv("/home/jovyan/work/slides/data/tweets.csv")
tweets

In [ ]:
import numpy as np

rows_no_coordinates = np.where(tweets['latitude']=="None")[0]

tweets_with_coordinates = tweets.drop(rows_no_coordinates)

tweets_with_coordinates

In [ ]:
tweets_with_coordinates.to_csv("/home/jovyan/work/slides/data/tweetCoordinates.csv",  index = False, header=True)

In [ ]:
# Intslamos gmplot
!pip install gmplot

In [ ]:
latitude = pd.to_numeric(tweets_with_coordinates["latitude"], downcast="float")
longitude = pd.to_numeric(tweets_with_coordinates["longitude"], downcast="float")

latitude

In [ ]:
import gmplot

# instancia
gmap = gmplot.GoogleMapPlotter(30,0,3)

#insertar puntos en el mapa pasando una lista de coordenadas
gmap.heatmap(latitude.tolist(), longitude.to_list(), radius=5)


#Guardar el mapa
gmap.draw('/home/jovyan/work/slides/heatmap.html')